In [1]:
import ccxt
import pandas as pd
from datetime import datetime, timedelta

# Initialize the exchange
exchange = ccxt.binance()

def fetch_historical_ohlcv(symbol, start_date, end_date):
    since = exchange.parse8601(start_date.isoformat())
    end_time = exchange.parse8601(end_date.isoformat())
    ohlcv_data = []

    while since < end_time:
        # Fetch the data
        data = exchange.fetch_ohlcv(symbol, timeframe='1d', since=since, limit=1000)
        if len(data) == 0:
            break
        ohlcv_data.extend(data)
        since = data[-1][0] + 86400000  # Move to the next day

    return ohlcv_data

def process_ohlcv_data(ohlcv_data):
    df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['average'] = df[['open', 'high', 'low', 'close']].mean(axis=1)
    df['barCount'] = df.shape[0]
    df = df[['date', 'open', 'high', 'low', 'close', 'average', 'barCount']]
    df.reset_index(drop=True, inplace=True)
    return df

# Define the cryptocurrencies and the date range
cryptocurrencies = [
    'ADA/USDT',
    'AUDIO/USDT',
    'AXS/USDT',
    'BTC/USDT',
    'CHZ/USDT',
    'DOT/USDT',
    'ETH/USDT',
    'FLOW/USDT',
    'LTC/USDT',
    'MANA/USDT',
    'SAND/USDT',
    'XRP/USDT'
]
end_date = datetime.now()
start_date = end_date - timedelta(days=90)

# Fetch and save data for each cryptocurrency
for symbol in cryptocurrencies:
    print(f"Fetching data for {symbol}...")
    ohlcv = fetch_historical_ohlcv(symbol, start_date, end_date)
    if not ohlcv:
        print(f"No data fetched for {symbol}")
        continue
    
    df = process_ohlcv_data(ohlcv)
    # Save to CSV
    csv_filename = symbol.replace("/", "_") + '_ohlcv_data.csv'
    df.to_csv(csv_filename, index=False)
    
print("Data fetched and saved successfully!")


Fetching data for BTC/USDT...
Fetching data for ETH/USDT...
Fetching data for LTC/USDT...
Fetching data for XRP/USDT...
Fetching data for ADA/USDT...
Fetching data for DOT/USDT...
Data fetched and saved successfully!


INDIVIDUAL CHART:

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from time import sleep
from lightweight_charts import Chart
import os

# Function to generate synthetic tick data based on OHLC data
def generate_ticks(ohlc_df):
    ticks = []
    
    for i, row in ohlc_df.iterrows():
        # Ensure date is in datetime format
        if isinstance(row['date'], str):
            start_time = pd.to_datetime(row['date'])
        else:
            start_time = row['date']
        
        end_time = start_time + timedelta(minutes=1)  # Assuming 1 minute interval

        # Generate random number of ticks (between 20 to 100 ticks per interval)
        num_ticks = random.randint(20, 100)
        
        # Generate synthetic tick prices
        tick_times = [start_time + (end_time - start_time) * i / num_ticks for i in range(num_ticks)]
        tick_prices = np.random.uniform(row['low'], row['high'], num_ticks)

        # Append tick data to the list
        for tick_time, tick_price in zip(tick_times, tick_prices):
            ticks.append({'time': tick_time.isoformat(), 'price': tick_price})

    # Create a DataFrame from the ticks
    ticks_df = pd.DataFrame(ticks)
    
    # Convert time to datetime, handling various formats
    ticks_df['time'] = pd.to_datetime(ticks_df['time'], format=None, errors='coerce')
    
    # Drop rows where conversion failed (if any)
    ticks_df = ticks_df.dropna(subset=['time'])
    
    # Sort by time
    ticks_df = ticks_df.sort_values(by='time').reset_index(drop=True)
    
    return ticks_df

if __name__ == '__main__':
    try:
        # Define the CSV filename and extract the symbol
        csv_filename = 'data/ADA_USDT_ohlcv_data.csv'
        symbol = os.path.basename(csv_filename).split('_')[0]  # Extract symbol from filename
        
        # Read OHLC data
        df1 = pd.read_csv(csv_filename)
        df1['date'] = pd.to_datetime(df1['date'])
        print("OHLC data loaded successfully.")
        
        # Generate synthetic tick data
        ticks_df = generate_ticks(df1)
        
        # Initialize and set up the chart
        chart = Chart()
        
        # Apply styling
        chart.layout(background_color='#090008', text_color='#FFFFFF', font_size=16, font_family='Helvetica')
        chart.candle_style(
            up_color='#00ff55', 
            down_color='#ed4807',
            border_up_color='#FFFFFF', 
            border_down_color='#FFFFFF',
            wick_up_color='#FFFFFF', 
            wick_down_color='#FFFFFF'
        )
        chart.volume_config(up_color='#00ff55', down_color='#ed4807')
        # Update watermark to include symbol and interval
        chart.watermark(f'1D {symbol}', color='rgba(180, 180, 240, 0.7)')
        
        # Configure crosshair
        chart.crosshair(
            mode='normal', 
            vert_color='#FFFFFF', 
            vert_style='dotted',
            horz_color='#FFFFFF', 
            horz_style='dotted'
        )
        
        # Configure legend (basic customization)
        chart.legend(visible=True, font_size=16, font_family='Arial')  # Changed font_size to 16 and font_family to 'Arial'

        # Add OHLC data to the chart
        chart.set(df1)
        chart.show()
        
        # Update the chart with synthetic tick data
        last_bar_time = df1['date'].max()  # Get the timestamp of the last OHLC bar
        
        for i, tick in ticks_df.iterrows():
            tick_time = tick['time']
            
            # Ensure tick timestamp is strictly after the last OHLC bar timestamp
            if tick_time > last_bar_time:
                chart.update_from_tick(tick)
                sleep(0.03)  # Control the speed of tick updates
        
        # Keep the script running to view the chart
        input("Press Enter to exit...")
        
    except Exception as e:
        print(f"An error occurred: {e}")


OHLC data loaded successfully.
Generated 430 synthetic ticks.
Last OHLC bar time: 2024-07-27 00:00:00
Updated tick: time     2024-07-27 00:00:15
price               0.424955
Name: 427, dtype: object
Updated tick: time     2024-07-27 00:00:30
price               0.414558
Name: 428, dtype: object
Updated tick: time     2024-07-27 00:00:45
price               0.422517
Name: 429, dtype: object


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import os
from lightweight_charts import Chart, LineSeries, HistogramSeries

# Function to generate synthetic tick data based on OHLC data
def generate_ticks(ohlc_df):
    ticks = []
    
    for i, row in ohlc_df.iterrows():
        if isinstance(row['date'], str):
            start_time = pd.to_datetime(row['date'])
        else:
            start_time = row['date']
        
        end_time = start_time + timedelta(minutes=1)

        num_ticks = random.randint(20, 100)
        
        tick_times = [start_time + (end_time - start_time) * i / num_ticks for i in range(num_ticks)]
        tick_prices = np.random.uniform(row['low'], row['high'], num_ticks)

        for tick_time, tick_price in zip(tick_times, tick_prices):
            ticks.append({'time': tick_time.isoformat(), 'price': tick_price})

    ticks_df = pd.DataFrame(ticks)
    ticks_df['time'] = pd.to_datetime(ticks_df['time'], format=None, errors='coerce')
    ticks_df = ticks_df.dropna(subset=['time'])
    ticks_df = ticks_df.sort_values(by='time').reset_index(drop=True)
    
    return ticks_df

def add_chart_to_pane(chart, df, symbol, pane_index):
    pane = chart.add_pane(pane_index)
    pane.title = symbol
    
    chart.add_candle_series(pane_index, data=df[['date', 'open', 'high', 'low', 'close']])
    chart.add_line_series(pane_index, data=df[['date', 'average']])
    chart.add_histogram_series(pane_index, data=df[['date', 'volume']])

def setup_charts():
    chart = Chart()
    
    # Define the layout
    chart.layout(
        background_color='#090008',
        text_color='#FFFFFF',
        font_size=16,
        font_family='Helvetica'
    )
    
    # Add chart styling
    chart.candle_style(
        up_color='#00ff55',
        down_color='#ed4807',
        border_up_color='#FFFFFF',
        border_down_color='#FFFFFF',
        wick_up_color='#FFFFFF',
        wick_down_color='#FFFFFF'
    )
    
    chart.volume_config(
        up_color='#00ff55',
        down_color='#ed4807'
    )
    
    chart.crosshair(
        mode='normal',
        vert_color='#FFFFFF',
        vert_style='dotted',
        horz_color='#FFFFFF',
        horz_style='dotted'
    )
    
    return chart

if __name__ == '__main__':
    try:
        data_dir = 'data/'
        csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
        
        chart = setup_charts()
        
        for index, csv_file in enumerate(csv_files):
            if index >= 6:  # Limit to 6 charts for a 2x3 grid
                break
            
            symbol = os.path.basename(csv_file).split('_')[0]
            df = pd.read_csv(os.path.join(data_dir, csv_file))
            df['date'] = pd.to_datetime(df['date'])
            df['average'] = df[['open', 'high', 'low', 'close']].mean(axis=1)
            
            # Generate synthetic tick data
            ticks_df = generate_ticks(df)
            
            # Add chart to pane
            add_chart_to_pane(chart, df, symbol, index)
        
        chart.show(block=True)
        
    except Exception as e:
        print(f"An error occurred: {e}")


ImportError: cannot import name 'LineSeries' from 'lightweight_charts' (c:\Users\Me\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightweight_charts\__init__.py)